In [19]:
# import dependencies
import pandas as pd
from collections import defaultdict
from pathlib import Path
import nltk as nl
from nltk.tokenize import word_tokenize
import re
from nltk.tokenize.casual import TweetTokenizer
import numpy as np
import math
import os 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
nl.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\600846\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
# pass path as train and test folder
def main():
    # print command line arguments
    train= get_data(sys.argv[1])
    test= get_data(sys.argv[2])

    clean_train_stem,clean_train_nostem= clean(train)
    clean_test_stem, clean_test_nostem= clean(test)
    
    #Binary and Freq representations for stemmed and nonstemmed data
    trainvector_stem_bin,trainvector_stem_freq,stem_vocab= getrep(clean_train_stem)
    trainvector_nostem_bin,trainvector_nostem_freq, nostem_vocab = getrep(clean_train_nostem)
    
     #calling naive bayes and logistic regression 4 times each to train on the 4 data vectors above
    #Naive Bayes
    result1 = Naivebayes(clean_train_stem, [0,1], stem_vocab,trainvector_stem_bin)
    result2 = Naivebayes(clean_train_stem, [0,1], stem_vocab,trainvector_stem_freq)
    result3 = Naivebayes(clean_train_nostem, [0,1], nostem_vocab,trainvector_nostem_bin)
    result4 = Naivebayes(clean_train_nostem, [0,1], nostem_vocab,trainvector_nostem_freq)
    
    #Logistic Regression
    result5 = logisticreg(clean_train_stem, [0,1], freqVector,bigdoc)
    result6 = logisticreg(clean_train_stem, [0,1], freqVector,bigdoc)
    result7 = logisticreg(clean_train_stem, [0,1], freqVector,bigdoc)
    result8 = logisticreg(clean_train_stem, [0,1], freqVector,bigdoc)

In [21]:
#calling test for each of the  models created: Print the accuracy and confusion matrix for each of them
if __name__ == "__main__":
    main()

NameError: name 'sys' is not defined

In [23]:
def get_data(path):
    my_dir_path_pos = path + '/positive'
    # create list to store text
    results = defaultdict(list)

    # loop through files and append text to list
    for file in Path(my_dir_path).iterdir():
        with open(file, "r", encoding="utf8") as file_open:
            results["text"].append(file_open.read())

    # read the list in as a dataframe
    df_pos = pd.DataFrame(results)

    # take a look at dataframe
    df_pos.head()

    #set directory path
    my_dir_path_neg = path + '/negative'

    # create list to store text
    results_neg = defaultdict(list)

    # loop through files and append text to list
    for file in Path(my_dir_path_neg).iterdir():
        with open(file, "r", encoding="utf8") as file_open:
            results_neg["text"].append(file_open.read())
    # read the list in as a dataframe
    df_neg = pd.DataFrame(results_neg)
    df_neg.head()

    #add sentiment to both datasets and then combine them for test data 1 for positive and 0 for negative
    df_pos['Sentiment']=1
    df_neg['Sentiment']=0
    frames = [df_pos, df_neg]
    df = pd.concat(frames)

    # increase column width to see more of the tweets
    pd.set_option('max_colwidth', 140)

    # reshuffle the tweets to see both pos and neg in random order
    df = df.sample(frac=1).reset_index(drop=True)

    # explore top 5 rows
    df.head(5)

    return df

In [24]:
def clean(df):

    # Remove any markup tags (HTML), all the mentions of handles(starts with '@') and '#' character
    def cleantweettext(raw_html):
        pattern = re.compile('<.*?>')
        cleantext = re.sub(pattern, '', raw_html)
        cleantext = " ".join(filter(lambda x:x[0]!='@', cleantext.split()))
        cleantext = cleantext.replace('#', '')
        return cleantext

    def removeat(text):
        atlist=[]
        for word in text:
            pattern = re.compile('^@')
            if re.match(pattern,word):
                #cleantext1 = re.sub(pattern, word[1:], word)
                atlist.append(word[1:])
            else:
                atlist.append(word)
        return atlist

    def tolower(text):
        lowerlist=[]
        for word in text:
            pattern = re.compile('[A-Z][a-z]+')
            if re.match(pattern,word):
                cleantext1 = re.sub(pattern, word.lower(), word)
                lowerlist.append(cleantext1)
            else:
                lowerlist.append(word)
        return lowerlist

    cleantweet= []
    for doc in df.text:
        cleantweet.append(cleantweettext(doc))


    tokentweet=[]
    df.text= cleantweet
    for doc in df.text:
        tokentweet.append(TweetTokenizer().tokenize(doc))
    df.text= tokentweet

    removeattweet=[]
    for doc in df.text:
        removeattweet.append(removeat(doc))
    df.text =removeattweet

    lowertweet=[]
    for doc in df.text:
        lowertweet.append(tolower(doc))
    df.text = lowertweet

    tweets=[]
    for x in df.text:
        tweet = ''
        for word in x:
            tweet += word+' '
        tweets.append(word_tokenize(tweet))
    df.text= tweets

    #stemming
    stemtweets=[]
    from nltk.stem.snowball import SnowballStemmer
    stemmer = SnowballStemmer("english", ignore_stopwords=False)
    #ps= PorterStemmer()
    for x in df.text:
        stemtweet=''
        for word in x:
            stemtweet=stemtweet+stemmer.stem(word)+' '
        stemtweets.append(word_tokenize(stemtweet))
    df['stemmed']=stemtweets

    ### Finalize both the stemmed and unstemmed dataframes
    df_unstemmed = df.drop(['stemmed'], axis=1)
    df_unstemmed.head()

    # create a df with stemmed text
    df_stemmed = df.drop(['text'], axis=1)

    return [df_stemmed,df_unstemmed]

In [25]:
def getrep()

SyntaxError: invalid syntax (<ipython-input-25-cbffc6ad82cd>, line 1)

In [26]:
# initialize count vectorizer
def dummy_fun(doc):
    return doc

def InitializeVectorization(text, kind):
    if kind == 'binary':
        vectorizer = CountVectorizer(binary = True, analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None)  
    else:
        vectorizer = CountVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None)  
    return vectorizer
    
def InitializeVocab(vectorizer, text):
    vectorizer.fit(text)
    freqVocab = vectorizer.vocabulary_
    train_vector = vectorizer.transform(text)
    len(freqVocab)
    return freqVocab

In [27]:
# Frequency - No stemming

vectorizer = VectorizationProc(df['text'], 'binary')
freqVocab = InitializeVocab(vectorizer, df['text'])

NameError: name 'VectorizationProc' is not defined

In [29]:
#Create bigdoc that contains words in V, their corresponding frequencies for each class

#1.Transform pos and neg tweets into seprate vectors
train_pos_vector1 = vectorizer.transform(df[df['Sentiment']==1]['text'])
train_neg_vector1 = vectorizer.transform(df[df['Sentiment']==0]['text'])

#2. column sum of vectors(word per column)
sum_pos = train_pos_vector1.sum(axis = 0)
sum_neg = train_neg_vector1.sum(axis = 0)

#3. Initialize bigdoc as a dataframe
bigdoc = pd.DataFrame(index = list(set(freqVocab.keys())), columns = ['pos', 'neg'])

#4. get the corresponding frequency from the above matrx and set it to bigdoc
for word in freqVocab.keys():
    index = freqVocab.get(word)
    bigdoc.at[word, 'pos'] = sum_pos[:, index].item()
    bigdoc.at[word, 'neg'] = sum_neg[:, index].item()

NameError: name 'vectorizer' is not defined

In [30]:
def Naivebayes(data,category,vector,bigvec):
    logprob = bigvec.copy()
    priors = []
    for cat in category:
        ndoc= len(data)
        nc= len(data[data['Sentiment']== cat])
        prior = nc/ndoc
        priors.append(prior)
        if cat == 0:
            colname = 'neg'
        else:
            colname = 'pos'
        denominator = bigvec[colname].sum() + len(bigvec) #denominator for likelihood
        logprob[colname] = bigvec[colname].apply(lambda x:math.log((x+1)/denominator)) #likelihood
    return [logprob,priors]

In [31]:
def TestNaiveBayes(testdoc, logprior, loglikelihood, category, V):

    for cat in category:
        sum[c]  logprior[c]
        for each position i in testdoc
        word testdoc[i]
        if word 2 V
        sum[c] = sum[c]+ loglikelihood[word,c]

    return argmaxc sum[c]

SyntaxError: invalid syntax (<ipython-input-31-ebe18b1f75f2>, line 4)

In [32]:
def logisticreg():

    #logistic regression
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    #initialize coefficients to zero

    def compute_cost(X, y, theta):
        m = len(y)
        h = sigmoid(X @ theta)
        epsilon = 1e-5
        cost = (1/m)*(((-y).T @ np.log(h + epsilon))-((1-y).T @ np.log(1-h + epsilon)))
        return cost

    def gradient_descent(X, y, params, learning_rate, iterations):
        m = len(y)
        cost_history = np.zeros((iterations,1))

        for i in range(iterations):
            params = params - (learning_rate/m) * (X.T @ (sigmoid(X @ params) - y)) 
            cost_history[i] = compute_cost(X, y, params)

        return (cost_history, params)